In [9]:
from pathlib import Path

# ============================================================
# 1) INPUT / OUTPUT
# ============================================================
input_file = Path("DataSets/MiniSet.alb")        # <-- anpassen
output_file = Path("DataSets/MiniSet_Extended.alb")

lines = input_file.read_text().splitlines()

# Anzahl Tasks aus <number of tasks>
n_tasks = None
for i, line in enumerate(lines):
    if line.strip() == "<number of tasks>":
        n_tasks = int(lines[i + 1].strip())
        break
if n_tasks is None:
    raise ValueError("Could not find <number of tasks> section")

print("Number of tasks:", n_tasks)


# ============================================================
# 2) MANUELL: STATES / HIERARCHIE / TASK-STATE / SETUPS
# ============================================================

# States
states = {
    1: "state_1",
    2: "state_2",
    3: "state_3",
}

# Hierarchie: von current_state aus sind diese to_states "kompatibel" ohne Setup
# Beispiel-Kette: 1 > 2 > 3
hierarchy_allows = {
    1: [1, 2, 3],
    2: [2, 3],
    3: [3],
}

# Task -> State-Kategorie (Task-Typ)
# MUSS alle Tasks 1..n_tasks abdecken
task_state = {
    1: 1,
    2: 2,
    3: 3,
    4: 2,
    5: 3,
    6: 1,
    7: 2,
    8: 3,
    9: 3,
    10: 1,
}

# ------------------------------------------------------------
# State->State setup definition
# ------------------------------------------------------------
# DEFAULT_SETUP ist die "normale" Umrüstzeit, wenn kein Spezialfall greift.
# Wenn du willst, dass z.B. 1->2 oder 2->3 immer 1 kosten, setze DEFAULT_SETUP = 1
DEFAULT_SETUP = 0

# Explizite, teure Umrüstungen (gerichtet!)
EXPLICIT_SETUP = {
    (2, 1): 20,
    (3, 1): 30,
}


def setup_time_state_to_state(s_from: int, s_to: int) -> int:
    """
    Setup-Zeit für einen Wechsel von State s_from -> s_to.
    """
    if (s_from, s_to) in EXPLICIT_SETUP:
        return EXPLICIT_SETUP[(s_from, s_to)]
    if s_from == s_to:
        return 0
    # Wenn Hierarchie "kompatibel" bedeutet: kein Setup nötig
    if s_to in hierarchy_allows.get(s_from, [s_from]):
        return 0
    return DEFAULT_SETUP


# ============================================================
# 3) NEUE SECTIONS ERZEUGEN
# ============================================================
new_sections = []

# <number of states>
new_sections += ["<number of states>", str(len(states)), ""]

# <states>
new_sections.append("<states>")
for sid, name in states.items():
    new_sections.append(f"{sid} {name}")
new_sections.append("")

# <state hierarchy>
new_sections.append("<state hierarchy>")
for s in states:
    allowed = ",".join(str(x) for x in hierarchy_allows[s])
    new_sections.append(f"{s}:{allowed}")
new_sections.append("")

# <state transitions>
# (in deinem bisherigen Format: current_state,task -> to_state)
# ist redundant, weil to_state nur vom task abhängt, aber lassen wir so.
new_sections.append("<state transitions>")
for s in states:
    for t in range(1, n_tasks + 1):
        to_state = task_state[t]
        new_sections.append(f"{s},{t}:{to_state}")
new_sections.append("")

# ============================================================
# WICHTIG: Setup-Matrix als STATE->STATE, NICHT State->Task
# ============================================================

# <setup times state forward>
# Jetzt echte Matrix: s_from, s_to : setup
new_sections.append("<setup times state forward>")
for s_from in states:
    for s_to in states:
        st = setup_time_state_to_state(s_from, s_to)
        new_sections.append(f"{s_from},{s_to}:{st}")
new_sections.append("")

# <setup times state cycle>
# Falls du "cycle (last-to-first)" anders modellieren willst, könntest du
# hier eine andere Funktion verwenden. Aktuell identisch zu forward.
new_sections.append("<setup times state cycle>")
for s_end in states:
    for s_start in states:
        st = setup_time_state_to_state(s_end, s_start)
        new_sections.append(f"{s_end},{s_start}:{st}")
new_sections.append("")


# ============================================================
# 4) ALTE SECTION <setup times forward> STRIKT ENTFERNEN
# ============================================================
filtered_lines = []
i = 0
while i < len(lines):
    if lines[i].strip() == "<setup times forward>":
        i += 1
        while i < len(lines):
            s = lines[i].strip()
            if s.startswith("<") and s.endswith(">"):
                break
            i += 1
        continue
    filtered_lines.append(lines[i])
    i += 1


# ============================================================
# 5) IN DATEI EINFÜGEN (vor <end>, falls vorhanden)
# ============================================================
out_lines = []
inserted = False
for line in filtered_lines:
    if line.strip() == "<end>" and not inserted:
        out_lines.extend(new_sections)
        inserted = True
    out_lines.append(line)

if not inserted:
    out_lines.append("")
    out_lines.extend(new_sections)

output_file.write_text("\n".join(out_lines) + "\n")
print("Wrote:", output_file)


Number of tasks: 10
Wrote: DataSets\MiniSet_Extended.alb
